In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import os
%matplotlib inline

t = np.arange(0,1500)
x = np.sin(0.02*t)+ np.random.rand(1500) * 2

train,test = x[0:1000], x[1000:]

# convert into dataset data and label
def convertToDataset(data, step):
    #data = np.append(data,np.repeat(data[-1,],step))
    X, Y =[], []
    for i in range(len(data)-step):
        d=i+step  
        X.append(data[i:d,])
        Y.append(data[d,])
    return np.array(X), np.array(Y)

C:\Users\shadm\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\shadm\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\shadm\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\shadm\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [2]:
train_step = 10
test_step = 20

trainX,trainY =convertToDataset(train,train_step)
testX,testY =convertToDataset(test,test_step)

trainX = np.reshape(trainX, (trainX.shape[0],  trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0],testX.shape[1], 1))

In [3]:
print(trainX.shape)
print(testX.shape)

(990, 10, 1)
(480, 20, 1)


In [4]:
def build_model(sequence_length):
    model = Sequential()
    model.add(SimpleRNN(units=64, input_shape=(sequence_length, 1),  activation="tanh"))
    model.add(Dense(1))
    return model

In [5]:
model = build_model(10)
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 64)                4224      
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 4,289
Trainable params: 4,289
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.input

<tf.Tensor 'simple_rnn_input:0' shape=(?, 10, 1) dtype=float32>

In [7]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [8]:
# Directory where the checkpoints will be saved
checkpoint_dir = './notebook02_training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [10]:
history = model.fit(trainX,trainY, epochs=100, batch_size=16,
                    verbose=2, callbacks=[checkpoint_callback])

Epoch 1/100
990/990 - 0s - loss: 0.2417
Epoch 2/100
990/990 - 0s - loss: 0.2340
Epoch 3/100
990/990 - 1s - loss: 0.2319
Epoch 4/100
990/990 - 0s - loss: 0.2314
Epoch 5/100
990/990 - 1s - loss: 0.2329
Epoch 6/100
990/990 - 0s - loss: 0.2268
Epoch 7/100
990/990 - 1s - loss: 0.2221
Epoch 8/100
990/990 - 0s - loss: 0.2174
Epoch 9/100
990/990 - 0s - loss: 0.2203
Epoch 10/100
990/990 - 1s - loss: 0.2110
Epoch 11/100
990/990 - 0s - loss: 0.2170
Epoch 12/100
990/990 - 0s - loss: 0.2024
Epoch 13/100
990/990 - 0s - loss: 0.1988
Epoch 14/100
990/990 - 0s - loss: 0.1977
Epoch 15/100
990/990 - 0s - loss: 0.1879
Epoch 16/100
990/990 - 1s - loss: 0.1904
Epoch 17/100
990/990 - 0s - loss: 0.1835
Epoch 18/100
990/990 - 0s - loss: 0.1812
Epoch 19/100
990/990 - 0s - loss: 0.1777
Epoch 20/100
990/990 - 1s - loss: 0.1763
Epoch 21/100
990/990 - 1s - loss: 0.1777
Epoch 22/100
990/990 - 0s - loss: 0.1716
Epoch 23/100
990/990 - 0s - loss: 0.1646
Epoch 24/100
990/990 - 1s - loss: 0.1661
Epoch 25/100
990/990 - 1s

UnknownError: Failed to rename: ./notebook02_training_checkpoints\checkpoint.tmp59527d8d192e4f5eab0ee98e6fd97ef9 to: ./notebook02_training_checkpoints\checkpoint : Access is denied.
; Input/output error

In [11]:
tf.train.latest_checkpoint(checkpoint_dir)

'./notebook02_training_checkpoints\\ckpt_87'

In [12]:
model = build_model(20)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.compile(loss='mean_squared_error', optimizer='adam')

In [13]:
trainScore = model.evaluate(testX, testY, verbose=0)
print(trainScore)

0.8677210330963134
